In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

이제 그림 \ref{fig:bigStepCBV}와 \ref{fig:bigStepCBN}의 큰걸음 의미구조를 하스켈로 옮겨 보자.

In [2]:
type Nm = String -- 변수 이름은 문자열로
data Expr = Var Nm         -- x
          | Lam Nm Expr    -- (λx.e)
          | App Expr Expr  -- (e1 e2)
          deriving (Eq, Ord)

In [3]:
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x e) = showParen (p > 1) $
      showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2

import IHaskell.Display (latex)
instance IHaskellDisplay Expr where -- 노트북에 디스플레이하는 인스턴스 선언
    display e = display [latex $ "$"++show e++"$"]

In [4]:
Lam "x" (Lam "y" (App (Var "x") (Lam "z"  (Var "y"))))

In [5]:
-- 연산자 (.), ($) 및 역따옴표 중위 표현 활용 
Lam "x" . Lam "y" $ Var "x" `App` Lam "z" (Var "y")

In [6]:
type Env = [(Nm,Clos)] -- 값계산 환경
data Clos = Cl Expr Env deriving (Eq,Ord,Show) -- 클로저

In [7]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Clos) where
    show (x,cl) = x++"{\\mapsto}"++show cl
instance Show Clos where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "

instance {-# OVERLAPS #-} IHaskellDisplay Env where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Clos) where
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Clos where
    display e = display [latex $ "$"++show e++"$"]

In [8]:
env01 = [ ("f", Cl (Lam "z" $ Var "z") []) ]

In [9]:
env01

In [10]:
env02 = [ ( "x", Cl (Lam "z" $ Var "z")               []   ),
          ( "y", Cl (Lam "x" $ Var "f" `App` Var "x") env01) ]

In [11]:
env02

In [12]:
mapM display it -- 직전 실행한 값계산 환경의 각 대응 항목을 한줄씩 디스플레이

In [13]:
:type lookup

lookup :: forall a b. Eq a => a -> [(a, b)] -> Maybe b

In [14]:
lookup "y" env02 -- Maybe Env 디스플레이 인스턴스 선언 없으므로 텍스트 출력
lookup "z" env02 -- 환경에 없는 변수 참조

Just \langle \lambda x.f\;x,\{f{\mapsto}\langle \lambda z.z,\{\}\rangle \}\rangle

Nothing

In [15]:
lookupEnv :: Nm -> Env -> [Clos]
lookupEnv x env = case lookup x env of Nothing -> []
                                       Just cl -> [cl]

In [16]:
lookupEnv "y" env02 -- 리스트 원소 한줄씩 display되는데 한개이므로

In [17]:
lookupEnv "z" env02 -- 빈 리스트라 아무것도 display되지 않음

In [18]:
bigStepCBV :: Clos -> [Clos]
bigStepCBV (Cl (Var x)   env) = lookupEnv x env
bigStepCBV (Cl (Lam x e) env) = undefined -- 연습문제로 작성
bigStepCBV (Cl (App x e) env) = undefined -- 연습문제로 작성

In [19]:
bigStepCBV (Cl (Var "y") env02) -- 큰걸음 실행 결과 있음 (환경에 있는 변수)

In [20]:
bigStepCBV (Cl (Var "z") env02) -- 큰걸음 실행 결과 없음 (환경에 없는 변수)

In [21]:
-- bigStepCBV (Cl (Lam ...) ...) -- 연습문제로 작성한 코드 관련 테스트 작성

In [22]:
-- bigStepCBV (Cl (App ...) ...) -- 연습문제로 작성한 코드 관련 테스트 작성

여기까지 인자 먼저 값계산의 큰걸음 의미구조(그림 \ref{fig:bigStepCBV})를 하스켈로 옮긴 함수 `bigStepCBV`를 작성하여 테스트해 보았다면,
또 하나의 연습문제로 적용 먼저 값계산의 큰걸음 의미구조(그림 \ref{fig:bigStepCBN})를 하스켈로 옮긴 함수 `bigStepCBN`도 마찬가지 방법으로
작성하고 테스트해 볼 수 있을 것이다. 같은 클로저를 `bigStepCBN`으로 실행하면 정상적으로 실행되어 결과값이 있는데
`bigStepCBV`는 정상적으로 실행되지 않고 결과값이 없는 경우가 없을지 생각해 보고 있다면 그러한 예시를 구성해 테스트해 보라.
또한 같은 클로저를 `bigStepCBN`과 `bigStepCBV`로 실행했을 때 모두 정상 실행되지만 두 결과값이 똑같지는 않은 경우는 없는지
생각해 보고 있다면 그러한 예시를 구성해 테스트해 보라.

In [23]:
bigStepCBN :: Clos -> [Clos]
bigStepCBN (Cl (Var x)   env) = undefined -- 연습문제로 작성
bigStepCBN (Cl (Lam x e) env) = undefined -- 연습문제로 작성
bigStepCBN (Cl (App x e) env) = undefined -- 연습문제로 작성

In [24]:
-- 연습문제로 작성한 bigStepCBN에 대한 테스트

# 값계산 환경을 활용한 작은걸음 의미구조
\label{sec:evalCEK}
이 장을 시작하며 \ref{sec:evalSmallStep}절 앞부분에서는 줄임(reduction)과 값계산(evaluation)을 연관지어 이해하기 좋도록
타입없는 람다계산법의 값계산을 맥락적 의미구조로 설명하였다. 그런데, 이런 치환 기반의 의미구조를 그대로 구현한다면 불필요한 치환이
발생하는 단점을 있다. 이런 단점이 해소된 값계산 환경을 활용하는 방식의 큰걸음 의미구조를 \ref{sec:evalBigStep}절에서 살펴보았다.
환경을 활용하는 큰걸음 의미구조에 더 직접적으로 대응되는 작은걸음 의미구조도 생각해 볼 수 있다.
이러한 작은걸음 의미구조가 바로 CEK기계\cite{Felleisen86a}이다.\footnote{여기서는
    조금 단순화된 형태의 다음과 같은 하스켈 코드를 참고해 정리하였다.\\$\phantom{AAx}$
    \url{https://matt.might.net/articles/cek-machines/} }
참고로, CEK는 코드(Code), 환경(Enviroment), 나중에 할 일(Kontinuation)의 두문자어(acronym)이다.
코드는 환경은 이미 익숙한 람다식과 값계산 환경을 말하므로 \`\`나중에 할 일\'\'(영어: continuation, 독일어: Kontinuation)의 개념만 알면
CEK기계의 작동 원리를 이해할 수 있다.

우선 산술식으로 나중에 할 일의 개념을 알아보자.
산술식 $(\underline{3 + 4}) + (5 + 6)$에서 밑줄 친 부분을 지금 계산하고 있다면,
전체 식을 계산하기 위해 나중에 할 일은 $\bullet + (5 + 6)$처럼 값계산 맥락((evaluation context)과 같은 형태로 표현할 수 있다.
나중에 할 일 $\bullet + (5 + 6)$는 지금 진행하는 계산의 결과를 $\bullet$ 부분에 넣어서 완성되는 전체 식을 계산하라는 뜻이다.
조금 더 큰 산술식 $(1 + 2) + ((\underline{3 + 4}) + (5 + 6))$에서 마찬가지로
밑줄 친 부분을 지금 계산하고 있다면, 전체 식을 계산하기 위해 나중에 할 일을
한꺼번에 $(1 + 2) + (\bullet + (5 + 6))$처럼 표현할 수도 있고,
또 $\bullet + (5 + 6)$와 $(1 + 2) + \bullet$처럼 차례대로 두 개의 나중에 할 일로 나누어 표현할 수도 있을 것이다.
정리하면, 산술식에서 나중에 할 일($\kappa$)이란 일련의 값계산 맥락들로 표현할 수 있다. 그리고 일련의 나중에 할 일 가장
마지막은 항상 전체 식의 값을 나타내는 맥락인 $\bullet$으로 항상 끝마친다고 생각할 수 있으므로,
산술식에 대한 나중에 할 일($\kappa$)의 문법구조를
$\kappa ::= \mathcal{E},\kappa \mid \bullet$로 정의할 수 있다.

환경 기반의 의미구조에서는 람다식($e$)과 값계산 환경($\sigma$)를 짝지어 계산을 진행하므로,
CEK기계에서 \`\`나중에 할 일\'\'은 람다식에 대한 맥락($\mathcal{E}$)과 값계산 환경($\sigma$)를 짝지은
$\langle\mathcal{E},\sigma\rangle$와 같은 형태를 차례로 나열하여 표현한다.
또한, 나중에 할 일의 마지막은 계산하는 닫힌식 전체의 값을 나타내는 $\langle\bullet,\{\}\rangle$이므로,
CEK 기계에서 나중에 할 일($\kappa$)의 문법구조를
$\kappa ::= \langle\mathcal{E},\sigma\rangle,\kappa \mid \langle\bullet,\{\}\rangle$로 정의할 수 있다.
CEK기계는 
$\langle e_1,\sigma_1\rangle;\kappa_1 \;\longmapsto\; \langle e_2,\sigma_2\rangle;\kappa_2 \;\longmapsto\;\ldots$
와 같이 작은걸음으로 계산이 진행되며 $v;\langle\bullet,\{\}\rangle$의 형태일 때 $v$라는 값으로 계산이 정상 종료된 것으로 본다.

이제부터는 인자 먼저(call-by-value) 값계산과 적용 먼저(call-by-name) 값계산을 규정하는 CEK기계의 구체적인 작은걸음 규칙을 살펴보기로 하자.

## 인자 먼저 값계산 CEK기계
인자 먼저 값계산 CEK기계(CEK$_\mathrm{v}$)의 작은걸음 규칙은 아래와 같다.

\begin{align*}
\langle x,\sigma\rangle;\kappa
        &\longmapsto
\sigma(x);\kappa 
& \text{(\textsc{Var})}
\\
\langle e_1~e_2,\sigma\rangle;\kappa
        &\longmapsto
\langle e_1,\sigma\rangle;\langle \bullet~e_2,\sigma\rangle,\kappa
& \text{(\textsc{App}$_1$)}
\\
\langle \lambda x.e,\sigma_1\rangle;\langle \bullet~e_2,\sigma\rangle,\kappa
        &\longmapsto
\langle e_2,\sigma\rangle;\langle(\lambda x.e)\,\bullet,\sigma_1\rangle,\kappa
& \text{(\textsc{App}$_2$)}
\\
v_2;\langle(\lambda x.e)\,\bullet,\sigma_1\rangle,\kappa
        &\longmapsto
\langle e,\{x{\mapsto}v_2\}\sigma_1\rangle;\kappa
& \text{(\textsc{Beta}$_\mathrm{v}$)}
\end{align*}

\noindent
위의 네 가지 규칙을 좀더 자세히 설명하면 다음과 같다.

- 첫째, 변수에 대한 규칙(\textsc{Var})으로 한걸음 진행하면, 변수에 대응되는 값을 환경에서 참조한다.
- 둘째, 함수적용식의 함수 부분에 대한 규칙(\textsc{App}$_1$)으로 한걸음 진행하면,
  우선 왼쪽의 함수 부분($e_1$)부터 현재 값계산 환경($\sigma$)에서 지금부터 계산하고
  나중에 할 일 목록에 앞으로 오른쪽의 인자 부분($e_2$)의 계산을 진행하기 위한 항목($\langle\bullet\;e_2,\sigma\rangle$)을 추가해 놓는다.
- 셋째, 함수적용식의 인자 부분에 대한 규칙(\textsc{App}$_2$)으로 함수 부분의 계산이 끝난 값의 형태($\langle\lambda x.e,\sigma_1$)일 때
  나중의 할 일 목록에서 가장 앞에 위치한 $\langle\bullet\;e_2,\sigma\rangle$를 제거하며 한걸음 진행하면,
  지금부터 함수 적용식의 인자 부분($e_2$)의 계산을 진행하되
  나중의 할 일 목록에 앞으로 함수 호출을 처리하기 위한 항목($\langle(\lambda x.e)\,\bullet,\sigma_1\rangle$)을 추가해 놓는다.
- 넷째, 함수 호출을 처리하는 규칙(\textsc{Beta}$_\mathrm{v}$)으로 인자 부분의 계산이 끝난 값($v_2$)의 형태일 때 
  나중의 할 일 목록에서 가장 앞에 위치한 $\langle(\lambda x.e)\,\bullet,\sigma_1\rangle$를 제거하며 한걸음 진행하면,
  함수 부분의 값을 나타내는 클로저에 포함된 기존의 환경($\sigma_1$)에
  추가로 함수 파라메터($x$)에 인자 부분이 계산된 값($v_2$)을 대응시켜 확장한 새로운 환경에서
  지금부터 함수 몸체($e$)의 계산을 진행한다.

\noindent
참고로, 마지막의 넷째 규칙(\textsc{Beta}$_\mathrm{v}$)에 나타나는 $v_2$는 환경 기반의 인자 먼저 값계산에서 닫힌식의 계산이 완료된 형태의 클로저,
즉 $\langle\lambda x_2.e_2,\sigma_2\rangle$와 같은 형태이다.

## 적용 먼저 값계산 CEK기계
적용 먼저 값계산 CEK기계(CEK$_\mathrm{n}$)의 작은걸음 규칙은 아래와 같다.

\begin{align*}
\langle x,\sigma\rangle;\kappa
        &\longmapsto
\sigma(x);\kappa
& \text{(\textsc{Var})}
\\
\langle e_1~e_2,\sigma\rangle;\kappa
        &\longmapsto
\langle e_1,\sigma\rangle;\langle\bullet~e_2,\sigma\rangle,\kappa
& \text{(\textsc{App}$_1$)}
\\
\langle \lambda x.e,\sigma_1\rangle;\langle\bullet~e_2,\sigma\rangle,\kappa
        &\longmapsto
\langle e,\{x{\mapsto}\langle e_2,\sigma\rangle\}\sigma_1\rangle;\kappa
& \text{(\textsc{Beta}$_\mathrm{n}$)}
\end{align*}

\noindent
CEK$_\mathrm{n}$의 {\textsc{Var}}와 {\textsc{App}$_1$}규칙은 앞서 살펴본 CEK$_\mathrm{v}$와 마찬가지지만
함수 적용식의 인자 부분에 대한 계산과 함수 호출에 대한 규칙에서 차이가 있다.
CEK$_\mathrm{v}$에서는 함수적용식의 인자 부분($e_2$)에 대한 계산을 {\textsc{App}$_2$}규칙으로 먼저 진행하고 나서
{\textsc{Beta}$_v$}규칙에 따라 함수를 호출하는 반면, CEK$_\mathrm{n}$에서는 함수적용식의 함수 부분($e_1$)에 대한 계산이 끝나면
곧바로 {\textsc{Beta}$_n$}규칙으로 인자 부분($e_2$)의 계산을 진행하지 않고 그대로 클로저를 구성해 함수를 호출한다.

## CEK기계의 작은걸음 함수를 하스켈로 작성하기
CEK기계와 관련된 규칙에 나타나는 나중에 할 일(`Kont`)을 하스켈로 옮기기 위해 다음과 같이 선언한다.

In [25]:
type Kont = [(Ctx,Env)] -- 나중에 할 일을 Ctx와 Env의 순서쌍 리스트로 표현
data Ctx = Done | App1 Expr | App2 Expr -- 일반적인 값계산 맥락보다 단순

위에 선언된 데이터 타입 `Ctx`는 일반적인 값계산 맥락보다 단순하다.
일반적으로 값계산 맥락은 $e_1\,(\bullet\;e_2)$와 같이 구멍을 나타내는 $\bullet$이 두 겹 이상 안쪽으로 파고든 부분식의 위치에 나타날 수 있으므로 재귀적인 데이터 타입으로 선언해야 한다.
하지만 CEK기계의 계산 과정에서는 이렇게 여러 겹 안쪽에 구멍이 뚫린 하나의 복잡한 값계산 맥락 대신에 이를 두 개의 단순한 맥락 $\bullet\;e_2$와 $e_1\;\bullet$로 나누어 순차적으로 적용하는 방식으로 계산이 진행된다.
따라서 전체($\bullet$) 또는 함수적용식의 왼쪽($\bullet\;e$)이나 오른쪽($e\;\bullet)$ 이렇게 세 가지 단순한 맥락만을 고려하면 되므로 `Ctx`를 재귀적으로 정의할 필요가 없다.

In [26]:
instance Show Ctx where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ Done = showString "\\bullet "
  showsPrec p (App1 e2) = showParen (p > 9) $
      showString "\\bullet \\;" . showsPrec 10 e2
  showsPrec p (App2 e1) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\; \\bullet"

import IHaskell.Display (latex)
instance IHaskellDisplay Ctx where -- 노트북에 디스플레이하는 인스턴스 선언
    display ctx = display [latex $ "$"++show ctx++"$"]

In [27]:
App1 (Var "x" `App` Var "y")
App2 (Lam "x" $ Var "x")

In [28]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Kont where
    show k = intercalate ",\\," (map show k)
instance {-# OVERLAPS #-} Show (Clos,Kont) where
    show (cl,k) = show cl++";\\,"++show k
instance {-# OVERLAPS #-} Show (Ctx,Env) where
    show (ctx,env) = "\\langle "++show ctx++","++show env++"\\rangle "
instance {-# OVERLAPS #-} IHaskellDisplay Kont where
    display k = display [latex $ "$"++show k++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Clos,Kont) where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Ctx,Env) where
    display e = display [latex $ "$"++show e++"$"]

In [29]:
(Done,[]) :: (Ctx,Env)

In [30]:
k03 = [ (App1 (Var "f"),        env01),
        (App2 (Lam "x" $ Var "x"), []),
        (Done,                     []) ]
k03 :: Kont

In [31]:
e03 = Lam "y" (Var "y" `App` Var "f")
(Cl e03 env01, k03) :: (Clos,Kont) -- 클로저와 나머지 할 일의 순서쌍

지금까지 CEK기계를 하스켈로 옮기는 데데 필요한 타입을 선언하고
노트북 환경에서 CEK기계의 작은걸음 규칙에 나타난 것과 마찬가지로 출력되도록 클래스 인스턴스를 선언하였다.
이제 인자 먼저 값계산을 따르는 CEK$_\mathrm{v}$와 적용 먼저 값계산을 따르는 CEK$_\mathrm{n}$의
작은걸음 함수 `stepCEKv`와 `stepCEKn`을 작성해 보라.

In [32]:
stepCEKv :: (Clos,Kont) -> [(Clos,Kont)] -- CEKv 작은걸음 함수
stepCEKv _ = undefined -- 여러 개의 등식으로 나누어 작성해도 좋다

In [33]:
stepCEKn :: (Clos,Kont) -> [(Clos,Kont)] -- CEKn 작은걸음 함수
stepCEKn _ = undefined -- 여러 개의 등식으로 나누어 작성해도 좋다

\section*{요점정리}
* TODO
* TODO
* TODO

\section*{연습문제}
1. TODO
1. TODO
1. TODO

\section*{탐구과제}
1. TODO
1. TODO
1. TODO